# Graduate Project
### CSCI-E7 Spring 2017
#### Due Date: 170503
#### Name: Michael Heppler
#### TF: Alan Xie

### Imported Libraries

In [2]:
import requests
import json
import datetime as dt
import pandas
import sklearn
from sklearn.tree import DecisionTreeClassifier

## Fishing with Python

A simple app that collects and processes (Requests, JSON, Pandas) local weather and tide data from various API resources. Enter location and select dates for recommended favorable conditions or locations to fish. Enter fishing log information to discover patterns in the conditions, dates, and locations.

### Location + Time

Select the location and time we want to forecast.

In [3]:
station = '8447180' # 8447180 Sandwich, MA

begin_date = '20170503'
end_date = '20170510'

### Tides: NOAA

Collect tide data from NOAA CO-OPS API using `requests` and `json` modules.

In [4]:
# add date range and location to url string
tides_url = ('https://tidesandcurrents.noaa.gov/api/datagetter?product=predictions&application=NOS.COOPS.TAC.WL'+
            '&begin_date='+begin_date+
            '&end_date='+end_date+
            '&datum=MLLW&station='+station+
            '&time_zone=lst_ldt&units=english&interval=hilo&format=json')

# NOAA tides + currents
tides = requests.get(tides_url)

tide_dict = tides.json()

day_tides = []

for i in tide_dict["predictions"]:
    day_tides.append(i)

day_tides_df = pandas.DataFrame(day_tides)
day_tides_df

,t,type,v
0,2017-05-03 05:38,H,9.538
1,2017-05-03 12:00,L,-0.310
2,2017-05-03 18:20,H,8.733
3,2017-05-04 00:24,L,0.645
4,2017-05-04 06:41,H,9.132
5,2017-05-04 13:00,L,0.025
6,2017-05-04 19:23,H,8.677
7,2017-05-05 01:27,L,0.720
8,2017-05-05 07:45,H,8.864
9,2017-05-05 13:58,L,0.239


### Forecast: Dark Sky

Collect weather forecast data from Dark Sky API using `requests` and `json` modules.

In [5]:
# key
key = '0123456789abcdef9876543210fedcba'

# Sandwich, MA
location = '41.772030,-70.506247'

weather_url = ('https://api.darksky.net/forecast/'+key+'/'+
               location+'?exclude=currently,minutely,hourly,alerts,flags')

drksky_data = requests.get(weather_url)

day_weather = drksky_data.json()

forecast_file = ('forecast_'+begin_date+'.json')

with open(forecast_file, mode='w', encoding='utf-8') as f:
    json.dump(day_weather, f, indent=4)

print("Done! Dumped to JSON.")

Done! Dumped to JSON.


### Format Time: `datetime`

Various functions using the `datatime` module to format the UNIX timestamp and vice versa.

In [6]:
def formatTime(t,f):
    """
    Use datetime module to convert UNIX timestamp using format passed in parameter.
    """
    # datetime source: http://stackoverflow.com/a/3682808
    return dt.datetime.fromtimestamp(int(t)).strftime(f)

In [7]:
def makeDtObjFromStr(t):
    """
    Use datetime module to convert formatted timestamp string to datetime object.
    """
    # datetime source: http://stackoverflow.com/a/9637908
    return dt.datetime.strptime(t, '%Y-%m-%d %H:%M')

In [8]:
def getDatetimeObj(t):
    """
    Use datetime module to convert UNIX timestamp to datetime object.
    """
    # source credit: Raman Prasad... THANK YOU!!!
    return dt.datetime.fromtimestamp(int(t))

### Difference in Minutes between Tide and Sunrise/Sunset

Function to calculate difference in minutes between tide and sunrise/sunset.

In [9]:
def getMinutesDiff(tide_datetime, sun_timestamp):
    """
    Calculate difference in minutes between tide and sunrise/sunset.
    """
    
    sun_datetime = getDatetimeObj(sun_timestamp)
    
    time_diff = tide_datetime - sun_datetime

    minutes_diff = (time_diff.total_seconds() / 60)
    
    return int(minutes_diff)

### Minutes from break?

In [10]:
# iterate days in tides list
for day_time in day_tides:
    
    tide_datetime = makeDtObjFromStr(day_time['t'])
    
    tide_day = day_time['t'].split(' ')[0]
    tide_time = day_time['t'].split(' ')[1]
    
    if tide_time == '12:00':
        for dayz in day_weather['daily']['data']:
            if tide_day == formatTime(dayz['sunriseTime'],'%Y-%m-%d'):
                day_time.update({'diffTime':getMinutesDiff(tide_datetime,dayz['sunriseTime'])})
                print ('diffTime:',getMinutesDiff(tide_datetime,dayz['sunriseTime']))
    
    elif tide_time < '12:00':
        for dayz in day_weather['daily']['data']:
            if tide_day == formatTime(dayz['sunriseTime'],'%Y-%m-%d'):
                day_time.update({'diffTime':getMinutesDiff(tide_datetime,dayz['sunriseTime'])})
                print ('diffTime:',getMinutesDiff(tide_datetime,dayz['sunriseTime']))

    else:
        for dayz in day_weather['daily']['data']:
            if tide_day == formatTime(dayz['sunsetTime'],'%Y-%m-%d'):
                day_time.update({'diffTime':getMinutesDiff(tide_datetime,dayz['sunsetTime'])})
                print ('diffTime:',getMinutesDiff(tide_datetime,dayz['sunsetTime']))

diffTime: 1
diffTime: 383
diffTime: -83
diffTime: -311
diffTime: 65
diffTime: -404
diffTime: -21
diffTime: -247
diffTime: 130
diffTime: -347
diffTime: 37
diffTime: -186
diffTime: 192
diffTime: -293
diffTime: 90
diffTime: -129
diffTime: 249
diffTime: -245
diffTime: 137
diffTime: -78
diffTime: 299
diffTime: -200
diffTime: 179
diffTime: -32
diffTime: 344
diffTime: -158
diffTime: 218
diffTime: 11
diffTime: 386
diffTime: -117


### Merged Diff Data to Pandas

In [11]:
day_tides_df = pandas.DataFrame(day_tides)
day_tides_df

,diffTime,t,type,v
0,1,2017-05-03 05:38,H,9.538
1,383,2017-05-03 12:00,L,-0.310
2,-83,2017-05-03 18:20,H,8.733
3,-311,2017-05-04 00:24,L,0.645
4,65,2017-05-04 06:41,H,9.132
5,-404,2017-05-04 13:00,L,0.025
6,-21,2017-05-04 19:23,H,8.677
7,-247,2017-05-05 01:27,L,0.720
8,130,2017-05-05 07:45,H,8.864
9,-347,2017-05-05 13:58,L,0.239


### Dict to List

In [12]:
predict_list = []

# day tide index
dy = 0

for i in day_tides:
    # format for predict training
    dy_list = [dy,i['diffTime']]
    # add to predict training list
    predict_list.append(dy_list)
    dy += 1

print (predict_list)

[[0, 1], [1, 383], [2, -83], [3, -311], [4, 65], [5, -404], [6, -21], [7, -247], [8, 130], [9, -347], [10, 37], [11, -186], [12, 192], [13, -293], [14, 90], [15, -129], [16, 249], [17, -245], [18, 137], [19, -78], [20, 299], [21, -200], [22, 179], [23, -32], [24, 344], [25, -158], [26, 218], [27, 11], [28, 386], [29, -117]]


### Predict Fishing Forecast

In [13]:
# index, mins difference between tide sunrise/sunset
features = [[9999, 1], [9998, 383], 
            [9997, -83], [9996, -311], 
            [9995, 65], [9994, -404], 
            [9993, -21], [9992, -247]]
# good or bad?
labels = [1,0,1,0,1,0,1,0]

clf = sklearn.tree.DecisionTreeClassifier()

clf = clf.fit(features,labels)

d = {1:'Good +',0:'Bad -'}

# print results
print ('Fishing Forecast')
# tides index counter
tides_i = 0
# predict results for tides list
results = (clf.predict(predict_list))
for r in results:
    # store tide date + type
    tide_date = day_tides[tides_i]['t']
    tide_type = day_tides[tides_i]['type']
    tides_i += 1
    print ('Date: ',tide_date,'  Tide: ',tide_type,'  Prediction:', d[r])

Fishing Forecast
Date:  2017-05-03 05:38   Tide:  H   Prediction: Good +
Date:  2017-05-03 12:00   Tide:  L   Prediction: Bad -
Date:  2017-05-03 18:20   Tide:  H   Prediction: Good +
Date:  2017-05-04 00:24   Tide:  L   Prediction: Bad -
Date:  2017-05-04 06:41   Tide:  H   Prediction: Good +
Date:  2017-05-04 13:00   Tide:  L   Prediction: Bad -
Date:  2017-05-04 19:23   Tide:  H   Prediction: Good +
Date:  2017-05-05 01:27   Tide:  L   Prediction: Bad -
Date:  2017-05-05 07:45   Tide:  H   Prediction: Good +
Date:  2017-05-05 13:58   Tide:  L   Prediction: Bad -
Date:  2017-05-05 20:23   Tide:  H   Prediction: Good +
Date:  2017-05-06 02:27   Tide:  L   Prediction: Bad -
Date:  2017-05-06 08:46   Tide:  H   Prediction: Good +
Date:  2017-05-06 14:53   Tide:  L   Prediction: Bad -
Date:  2017-05-06 21:17   Tide:  H   Prediction: Good +
Date:  2017-05-07 03:22   Tide:  L   Prediction: Good +
Date:  2017-05-07 09:41   Tide:  H   Prediction: Bad -
Date:  2017-05-07 15:42   Tide:  L   Pr